In [78]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

In [79]:
error_lines_bad = False
data = pd.read_csv("clean_data.csv")
data

,Unnamed: 0,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee,department
0,0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal themes in American culture taught by an intercoll...,HarveyMudd,300,['Staff'],NaN,NaN,NaN,False,0,AMST
1,1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print culture through the careful examination of both t...,HarveyMudd,300,['Anup Gampa'],NaN,NaN,NaN,True,0,AMST
2,2,Hyphenated Americans,AMST-120-HM,A focus on the experience of immigrants in the United States and Americans of diverse ethnic bac...,HarveyMudd,300,['Balseiro'],NaN,NaN,NaN,False,0,AMST
3,3,"Life: Knowledge, Belief, and Cultural Practices",ANTH-110-HM,An exploration of cultural attitudes toward life and the human body: from Melanesian origin myth...,HarveyMudd,300,['de Laet'],NaN,NaN,NaN,False,0,ANTH
4,4,Introduction to the Anthropology of Science and Technology,ANTH-111-HM,An introduction to science and technology as cultural phenomena and a hands-on initiation into a...,HarveyMudd,300,['Marianne De Laet'],NaN,NaN,NaN,True,0,ANTH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919,4147,Advanced Academic Writing,WRIT-080-PZ,An advanced course in using sources to develop original scholarly arguments. To make discussions...,Pitzer,0,[],NaN,NaN,NaN,False,0,WRIT
3920,4148,Teaching and Tutoring of Writing,WRIT-100-PZ,"“Good writers are born, not made” - so the common misconception goes. Yet writing is a deeply so...",Pitzer,0,[],NaN,NaN,NaN,False,0,WRIT
3921,4149,Writing Center Theory and Praxis,WRIT-100A-PZ,"“Good writers are born, not made” - so the common misconception goes. Yet research shows us writ...",Pitzer,0,[],NaN,NaN,NaN,False,0,WRIT
3922,4150,Rhetoric and Argument,WRIT-115-PZ,A course for students interested in argumentation and the rhetorical analysis of articles and sp...,Pitzer,0,[],NaN,NaN,NaN,False,0,WRIT


In [80]:
data['identifier'] = [x.split('-')[0] for x in data['identifier']] # new data set has a department column that can be used instead
data

,Unnamed: 0,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee,department
0,0,Introduction to American Cultures,AMST,An interdisciplinary introduction to principal themes in American culture taught by an intercoll...,HarveyMudd,300,['Staff'],NaN,NaN,NaN,False,0,AMST
1,1,Print and American Culture,AMST,Covers numerous developments in American print culture through the careful examination of both t...,HarveyMudd,300,['Anup Gampa'],NaN,NaN,NaN,True,0,AMST
2,2,Hyphenated Americans,AMST,A focus on the experience of immigrants in the United States and Americans of diverse ethnic bac...,HarveyMudd,300,['Balseiro'],NaN,NaN,NaN,False,0,AMST
3,3,"Life: Knowledge, Belief, and Cultural Practices",ANTH,An exploration of cultural attitudes toward life and the human body: from Melanesian origin myth...,HarveyMudd,300,['de Laet'],NaN,NaN,NaN,False,0,ANTH
4,4,Introduction to the Anthropology of Science and Technology,ANTH,An introduction to science and technology as cultural phenomena and a hands-on initiation into a...,HarveyMudd,300,['Marianne De Laet'],NaN,NaN,NaN,True,0,ANTH
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919,4147,Advanced Academic Writing,WRIT,An advanced course in using sources to develop original scholarly arguments. To make discussions...,Pitzer,0,[],NaN,NaN,NaN,False,0,WRIT
3920,4148,Teaching and Tutoring of Writing,WRIT,"“Good writers are born, not made” - so the common misconception goes. Yet writing is a deeply so...",Pitzer,0,[],NaN,NaN,NaN,False,0,WRIT
3921,4149,Writing Center Theory and Praxis,WRIT,"“Good writers are born, not made” - so the common misconception goes. Yet research shows us writ...",Pitzer,0,[],NaN,NaN,NaN,False,0,WRIT
3922,4150,Rhetoric and Argument,WRIT,A course for students interested in argumentation and the rhetorical analysis of articles and sp...,Pitzer,0,[],NaN,NaN,NaN,False,0,WRIT


In [81]:
#agg_functions = {'identifier': 'first', 'title':'sum', 'description': 'sum'}
#pd.set_option('display.max_colwidth', 100)
data_major = data.groupby(data['identifier'])['description'].apply(lambda x: ','.join(x)).reset_index()
data_major.head(5)

,identifier,description
0,AFRI,"This class will serve as a general introduction to Africana Studies. Africana studies, while sti..."
1,AMST,An interdisciplinary introduction to principal themes in American culture taught by an intercoll...
2,ANTH,An exploration of cultural attitudes toward life and the human body: from Melanesian origin myth...
3,ARBC,Introductory Arabic is designed for students who do not have any Arabic language background. Emp...
4,ARBT,Offers a window into the richness and sophistication of modern Arabic poetry. It focuses on majo...


In [82]:
#data["instructors"]

In [83]:
pd.set_option('display.max_rows', 25)
data['instructors'] = [''.join(x) for x in data['instructors']]
#data

In [84]:
#agg_functions = {'instructors': 'first', 'title':'sum', 'description': 'sum'}
data_prof = data.groupby(data['instructors'])['description'].apply(lambda x: ','.join(x)).reset_index()
data_prof.head(5)

,instructors,description
0,"[""Harmony O'Rourke""]",This course will provide an introduction to the history of Africa from the build-up to European ...
1,"[""Melissa E. O'Neill"", 'Erin Talvitie']",Abstract data types including priority queues and dynamic dictionaries and efficient data struct...
2,"[""Melissa E. O'Neill""]","Oral presentations and discussions of selected topics, including recent developments in computer..."
3,"[""Michael D. O'Neill""]","Countable sets, least upper bound, and metric space topology including compactness, completeness..."
4,"[""Michael O'Malley""]","This is a beginning level wood sculpture course devoted to the knowledge of wood, the tools used..."


### Tf-Idf

In [85]:
import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [86]:
def tfidf(word, data):
    corpus = list(data.identifier)
    vectorizer = TfidfVectorizer(use_idf=True)
    vectors = vectorizer.fit_transform(corpus)

    score_for_word = []
    words = vectorizer.get_feature_names()
    try:
        index = words.index(word)
    except:
        print("'" + word + "'" + " is not mentioned in any course descriptions")
        return

    for i in range(0, len(corpus)):
        value = vectors[i].T.todense()[index]
        score_for_word.append(value)

    score_for_word = [float(i) for i in score_for_word] # type cast each score to a float

    data["score"] = score_for_word
    data = data.sort_values(by=["score"], ascending = False)
    return data

In [75]:
data_major.head()

,identifier,description
0,AFRI,"This class will serve as a general introduction to Africana Studies. Africana studies, while sti..."
1,AMST,An interdisciplinary introduction to principal themes in American culture taught by an intercoll...
2,ANTH,An exploration of cultural attitudes toward life and the human body: from Melanesian origin myth...
3,ARBC,Introductory Arabic is designed for students who do not have any Arabic language background. Emp...
4,ARBT,Offers a window into the richness and sophistication of modern Arabic poetry. It focuses on majo...


In [88]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
def cos_sim_id(data, id, penalty=1.0):
    corpus = list(data.description)
    vectorizer = TfidfVectorizer(use_idf=True, stop_words='english')
    vectors = vectorizer.fit_transform(corpus)
    cosine_sim = linear_kernel(vectors, vectors)

    index_id = list(data.index[data["identifier"] == id]) # indexes of input classes
    if len(index_id) == 0:
        print("Couldn't find courses")
        return
    else:
        index_id = index_id[0]
    
    data_out = data.copy()
    data_out["score"] = cosine_sim[index_id]

    if penalty!=1.0:
        department = id[:id.index("-")]
        print("penalizing " + department +" courses by a factor of " + str(penalty))
        
        for index in data_out.index:
            if data_out["identifier"][index] == id:
                data_out["score"][index] = 0.0
            elif department in data_out["identifier"][index]:
                data_out["score"][index] = data_out["score"][index]*penalty
    else:
        print("No penalty")

    return data_out



In [90]:
cos_sim_id(data_major, 'CSCI').sort_values(by=["score"], ascending = False).head(10)

No penalty


,identifier,description,score
25,CSCI,Introduction to elements of computer science. Students learn computational problem-solving techn...,1.000000
29,DS,This course allows teams of students to wrestle with real-world problems in data science. Studen...,0.372073
34,ENGR,"Design problems are, typically, open-ended and ill-structured. Students work in small teams appl...",0.324863
69,MATH,A comprehensive view of the theory and techniques of differential and integral calculus of a sin...,0.320539
67,LGCS,What exactly is language? What do you actually know when you know a language? These questions ar...,0.278725
12,BIOL,"Application of molecular biology techniques to problems in human genetics, bioengineering, and e...",0.270095
88,PSYC,An introduction to the field of psychology with a special emphasis on overarching themes and met...,0.254372
77,NEUR,An introduction to the nervous system and behavior that explores fundamental issues in neuroscie...,0.219906
74,MS,"This course will examine representations of animals in film - wildlife documentaries, animated f...",0.219405
31,ECON,An introductory course designed to provide a fundamental understanding of the national economy. ...,0.214259


In [91]:
data_prof.head()

,instructors,description
0,"[""Harmony O'Rourke""]",This course will provide an introduction to the history of Africa from the build-up to European ...
1,"[""Melissa E. O'Neill"", 'Erin Talvitie']",Abstract data types including priority queues and dynamic dictionaries and efficient data struct...
2,"[""Melissa E. O'Neill""]","Oral presentations and discussions of selected topics, including recent developments in computer..."
3,"[""Michael D. O'Neill""]","Countable sets, least upper bound, and metric space topology including compactness, completeness..."
4,"[""Michael O'Malley""]","This is a beginning level wood sculpture course devoted to the knowledge of wood, the tools used..."


In [93]:
def cos_sim_prof(data, prof):
    corpus = list(data.description)
    vectorizer = TfidfVectorizer(use_idf=True, stop_words='english')
    vectors = vectorizer.fit_transform(corpus)
    cosine_sim = linear_kernel(vectors, vectors)

    index_id = list(data.index[data["instructors"].str.contains(prof)]) # indexes of input classes
    if len(index_id) == 0:
        print("Couldn't find courses")
        return
    else:
        index_id = index_id[0]
    
    data_out = data.copy()
    data_out["score"] = cosine_sim[index_id]

    return data_out



In [94]:
cos_sim_prof(data_prof, "Harmony O'Rourke").sort_values(by=["score"], ascending = False).head(10)

,instructors,description,score
0,"[""Harmony O'Rourke""]",This course will provide an introduction to the history of Africa from the build-up to European ...,1.000000
21,['A. Mayes'],Examines the rise of the Aztec and Incan empires; the Spanish conquest and settlement of the Ame...,0.228969
876,['P. Chu'],This seminar explores connections between the history of science and European colonial expansion...,0.223855
1189,[],"This class will serve as a general introduction to Africana Studies. Africana studies, while sti...",0.222671
881,['P. Jackson'],Interdisciplinary seminar explores the ascension of intersectional feminism(s) produced by trail...,0.221178
1087,['T. Summers Sandoval'],‘Digital storytelling’ is the practice of everyday people who use digital tools to tell their st...,0.178768
438,['H. Wall'],"Development of the United States from colonial times to the late 19th century, emphasizing the s...",0.175528
53,['Albert Dorsey'],Recognizing the diverse voices and experiences of people of African descent in the United States...,0.166647
759,['M.O. Traoré'],History of Africa from the earliest times to the beginning of the 19th century. Attention given ...,0.166416
1038,['Sikivu Hutchinson'],"Although completely denied by dominant national narratives, the United States is a settler colon...",0.160664


# SKIP

In [51]:
# Checking shape of the data
data.shape

(4443, 13)

In [35]:
# Checking dataset
data.head(2)

,title,identifier,description,source,credits,instructors,offered,prerequisites,corequisites,currently_offered,fee,index_col,index
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...,HarveyMudd,300,[Staff],,,,False,0,0,1
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...,HarveyMudd,300,[Anup Gampa],,,,True,0,1,2


In [36]:
# Checking list of variables in the data
data.columns

Index(['title', 'identifier', 'description', 'source', 'credits',
       'instructors', 'offered', 'prerequisites', 'corequisites',
       'currently_offered', 'fee', 'index_col', 'index'],
      dtype='object')

In [37]:
# Creating data with important features
imp_features = ['title', 'identifier', 'description', 'source']
data1 = data[imp_features]
data1.head()

,title,identifier,description,source
0,Introduction to American Cultures,AMST-103-HM,An interdisciplinary introduction to principal...,HarveyMudd
1,Print and American Culture,AMST-115-HM,Covers numerous developments in American print...,HarveyMudd
2,Hyphenated Americans,AMST-120-HM,A focus on the experience of immigrants in the...,HarveyMudd
3,"Life: Knowledge, Belief, and Cultural Practices",ANTH-110-HM,An exploration of cultural attitudes toward li...,HarveyMudd
4,Introduction to the Anthropology of Science an...,ANTH-111-HM,An introduction to science and technology as c...,HarveyMudd


In [38]:
# Checking missing values
data1.isnull().sum()

title          0
identifier     0
description    0
source         0
dtype: int64

In [39]:
# Replacing missing values
# not sure if this is needed
missing_var = data1.isnull().sum().keys()
for i in missing_var:
    data1[i] = data1[i].fillna('')

/var/folders/n4/wr2pjk454g5cxqdbd2qywndc0000gn/T/ipykernel_39126/2770977688.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[i] = data1[i].fillna('')


In [40]:
# Checking missing values
data1.isnull().sum()

title          0
identifier     0
description    0
source         0
dtype: int64

In [41]:
# Combining the values of the all features
data1['Combined_features'] = data1['title']+' '+data1['identifier']+' '+data1['description']+' '+data1['source']

/var/folders/n4/wr2pjk454g5cxqdbd2qywndc0000gn/T/ipykernel_39126/1213261718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Combined_features'] = data1['title']+' '+data1['identifier']+' '+data1['description']+' '+data1['source']


In [42]:
#Checking first five values of the combined features
data1['Combined_features'][0:5]

0    Introduction to American Cultures AMST-103-HM ...
1    Print and American Culture AMST-115-HM Covers ...
2    Hyphenated Americans AMST-120-HM A focus on th...
3    Life: Knowledge, Belief, and Cultural Practice...
4    Introduction to the Anthropology of Science an...
Name: Combined_features, dtype: object

In [43]:
#Creating an object of count victorizer
cv = CountVectorizer()

In [44]:
#Creating count vectorization of combined features
count_matrix = cv.fit_transform(data1['Combined_features'])

In [45]:
#Computing cosine similarity of count matrix
cosin_sim = cosine_similarity(count_matrix)

In [46]:
#Checking first five values cosine similarity
cosin_sim[5]

array([0.0862796 , 0.25951446, 0.5668485 , ..., 0.        , 0.        ,
       0.        ])

In [49]:
#Defining function for course recommendation system
def get_recommender(course_title):
    
    #Finding an index number of given movie title which you like most
    course_index = data[data.title==course_title]['index'].values[0]
    
    #Finding cosine similarity score with respect to an index number of given movie title which you like most
    similar_courses = list(enumerate(cosin_sim[course_index]))
    
    #Sorting cosine similarity scores in descending order
    sorted_similar_courses = sorted(similar_courses,key=lambda x:x[1], reverse=True)
    
    #Extracting index numbers from sorted cosine similarity scores
    courses_indices = [i[0] for i in sorted_similar_courses]
    
    #Extracting movies title with respect to extracted index numbers from sorted cosine similarity scores
    courses_name = data['title'].iloc[courses_indices]
    
    #Creating dataframe of recommended movies name and resettig the index number
    recommended_courses = pd.DataFrame()
    recommended_courses['Recommended Courses'] = courses_name
    recommended_courses = recommended_courses.reset_index(drop = True)
    
    #Showing the result
    return recommended_courses.iloc[0:10]

In [50]:
get_recommender('Linear Algebra')

,Recommended Courses
0,Differential Equations
1,Differential Equations
2,Differential Equations
3,Dynamical Systems
4,Dynamics of Rigid Bodies
5,Differential Equations and Modeling
6,Scientific Computing
7,Scientific Computing
8,Numerical Analysis
9,Computational Partial Differential Equations
